# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

---

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

---

# <center>Part 4.集成算法的训练与优化技巧

In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc
from joblib import dump, load
from sklearn.ensemble import VotingClassifier
from hyperopt import hp, fmin, tpe, Trials
from numpy.random import RandomState
from sklearn.model_selection import cross_val_score

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [3]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [4]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [5]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [6]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

然后是模型融合部分所需准备的数据以及训练好的模型：

In [7]:
# 实例化KFold评估器
kf = KFold(n_splits=5, random_state=12, shuffle=True)

# 重置训练集和测试集的index
X_train_OE = X_train_OE.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

train_part_index_l = []
eval_index_l = []

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    train_part_index_l.append(train_part_index)
    eval_index_l.append(eval_index)
    
# 训练集特征
X_train1 = X_train_OE.loc[train_part_index_l[0]]
X_train2 = X_train_OE.loc[train_part_index_l[1]]
X_train3 = X_train_OE.loc[train_part_index_l[2]]
X_train4 = X_train_OE.loc[train_part_index_l[3]]
X_train5 = X_train_OE.loc[train_part_index_l[4]]

# 验证集特征
X_eval1 = X_train_OE.loc[eval_index_l[0]]
X_eval2 = X_train_OE.loc[eval_index_l[1]]
X_eval3 = X_train_OE.loc[eval_index_l[2]]
X_eval4 = X_train_OE.loc[eval_index_l[3]]
X_eval5 = X_train_OE.loc[eval_index_l[4]]

# 训练集标签
y_train1 = y_train.loc[train_part_index_l[0]]
y_train2 = y_train.loc[train_part_index_l[1]]
y_train3 = y_train.loc[train_part_index_l[2]]
y_train4 = y_train.loc[train_part_index_l[3]]
y_train5 = y_train.loc[train_part_index_l[4]]

# 验证集标签
y_eval1 = y_train.loc[eval_index_l[0]]
y_eval2 = y_train.loc[eval_index_l[1]]
y_eval3 = y_train.loc[eval_index_l[2]]
y_eval4 = y_train.loc[eval_index_l[3]]
y_eval5 = y_train.loc[eval_index_l[4]]

train_set = [(X_train1, y_train1), 
             (X_train2, y_train2), 
             (X_train3, y_train3), 
             (X_train4, y_train4), 
             (X_train5, y_train5)]

eval_set = [(X_eval1, y_eval1), 
            (X_eval2, y_eval2), 
            (X_eval3, y_eval3), 
            (X_eval4, y_eval4), 
            (X_eval5, y_eval5)]

In [8]:
# 随机森林模型组
grid_RF_1 = load('./models/grid_RF_1.joblib') 
grid_RF_2 = load('./models/grid_RF_2.joblib') 
grid_RF_3 = load('./models/grid_RF_3.joblib') 
grid_RF_4 = load('./models/grid_RF_4.joblib') 
grid_RF_5 = load('./models/grid_RF_5.joblib') 

RF_1 = grid_RF_1.best_estimator_
RF_2 = grid_RF_2.best_estimator_
RF_3 = grid_RF_3.best_estimator_
RF_4 = grid_RF_4.best_estimator_
RF_5 = grid_RF_5.best_estimator_

RF_l = [RF_1, RF_2, RF_3, RF_4, RF_5]

# 决策树模型组
grid_tree_1 = load('./models/grid_tree_1.joblib')
grid_tree_2 = load('./models/grid_tree_2.joblib')
grid_tree_3 = load('./models/grid_tree_3.joblib')
grid_tree_4 = load('./models/grid_tree_4.joblib')
grid_tree_5 = load('./models/grid_tree_5.joblib')

tree_1 = grid_tree_1.best_estimator_
tree_2 = grid_tree_2.best_estimator_
tree_3 = grid_tree_3.best_estimator_
tree_4 = grid_tree_4.best_estimator_
tree_5 = grid_tree_5.best_estimator_

tree_l = [tree_1, tree_2, tree_3, tree_4, tree_5]

# 逻辑回归模型组
grid_lr_1 = load('./models/grid_lr_1.joblib')
grid_lr_2 = load('./models/grid_lr_2.joblib')
grid_lr_3 = load('./models/grid_lr_3.joblib')
grid_lr_4 = load('./models/grid_lr_4.joblib')
grid_lr_5 = load('./models/grid_lr_5.joblib')

lr_1 = grid_lr_1.best_estimator_
lr_2 = grid_lr_2.best_estimator_
lr_3 = grid_lr_3.best_estimator_
lr_4 = grid_lr_4.best_estimator_
lr_5 = grid_lr_5.best_estimator_

lr_l = [lr_1, lr_2, lr_3, lr_4, lr_5]

In [9]:
eval1_predict_proba_RF = pd.Series(RF_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_RF = pd.Series(RF_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_RF = pd.Series(RF_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_RF = pd.Series(RF_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_RF = pd.Series(RF_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_RF = pd.concat([eval1_predict_proba_RF, 
                                   eval2_predict_proba_RF, 
                                   eval3_predict_proba_RF, 
                                   eval4_predict_proba_RF, 
                                   eval5_predict_proba_RF]).sort_index()

eval1_predict_proba_tree = pd.Series(tree_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_tree = pd.Series(tree_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_tree = pd.Series(tree_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_tree = pd.Series(tree_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_tree = pd.Series(tree_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_tree = pd.concat([eval1_predict_proba_tree, 
                                     eval2_predict_proba_tree, 
                                     eval3_predict_proba_tree, 
                                     eval4_predict_proba_tree, 
                                     eval5_predict_proba_tree]).sort_index()

eval1_predict_proba_lr = pd.Series(lr_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_lr = pd.Series(lr_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_lr = pd.Series(lr_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_lr = pd.Series(lr_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_lr = pd.Series(lr_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_lr = pd.concat([eval1_predict_proba_lr, 
                                   eval2_predict_proba_lr, 
                                   eval3_predict_proba_lr, 
                                   eval4_predict_proba_lr, 
                                   eval5_predict_proba_lr]).sort_index()

In [10]:
test_predict_proba_RF = []

for i in range(5):
    test_predict_proba_RF.append(RF_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_RF = np.array(test_predict_proba_RF)
test_predict_proba_RF = test_predict_proba_RF.mean(0)

test_predict_proba_tree = []

for i in range(5):
    test_predict_proba_tree.append(tree_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_tree = np.array(test_predict_proba_tree)
test_predict_proba_tree = test_predict_proba_tree.mean(0)

test_predict_proba_lr = []

for i in range(5):
    test_predict_proba_lr.append(lr_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_lr = np.array(test_predict_proba_lr)
test_predict_proba_lr = test_predict_proba_lr.mean(0)

## <center>Ch.3 模型融合基础方法

- 算法工程师的工程化实践能力

&emsp;&emsp;在介绍了Stacking过程中一级学习器和元学习器的训练和优化策略后，本节我们将进一步介绍Stacking模型融合优化函数的创建和使用，即重点介绍如何工程化高效落地实践Stacking融合及其优化过程。在本小节中，我们将详细总结此前介绍的Stacking模型融合完整流程，并通过一系列函数和类的编写来将其进行封装整合，和简单的sklearn中的Stacking评估器不同，这部分内容函数的编写将融合此前所有的优化策略，以大幅提升Stacking融合结果。本部分编写的函数既是为实战过程补充“枪支弹药”，同时也是后续进阶优化——级联优化之必须。

&emsp;&emsp;自特征工程部分内容开始，课程中工程化实现的代码就开始大幅增加，这些手动编写的函数和类，不仅仅是为了补充现有第三方库功能上的不足、给大家提供更多趁手的工具，更是为了借此提升大家的工程化实践能力。

&emsp;&emsp;正所谓“纸上得来终觉浅，绝知此事要躬行”。其实，对于算法工程人员来说，精通一个个算法背后的数学原理和调用第三方库来实现是远远不够的。很多时候，工程化实现能力也会很大程度影响最终的建模效果。以模型融合为例，能否设计一个高效的融合流程、能否通过编写代码实现这个流程、能否借助这个流程来批量的测试和筛选最佳融合方案（模型组合、特征组合、策略组合等），都将很大程度影响最终能否获得一个更好的结果。当然由此其实也能看出，其实算法工程师的工程化能力不仅仅是代码能力，算法流程的设计能力也是非常重要的一环。本届开始，借助模型融合优化函数的编写和封装，我们也将开始逐渐介绍如何设计一套完整、合理、高效的算法流程。

- 提前准备manual_ensemble.py文件

&emsp;&emsp;和特征工程类似，接下来我们也将单独创建一个py文件，作为模型融合函数库，存储一系列模型融合优化函数。因此需要提前创建好manual_ensemble.py文件，其基本格式和features_creation.py一致。创建完成后，需要将此前定义的VotingClassifier_threshold类和train_cross函数写入。编写完成后，即可按照如下方式进行导入：

In [11]:
import manual_ensemble as me

In [12]:
me?

Type:        module
String form: <module 'manual_ensemble' from 'D:\\Work\\jupyter\\telco\\正式课程\\manual_ensemble.py'>
File:        d:\work\jupyter\telco\正式课程\manual_ensemble.py
Docstring:   自动化批量特征衍生模块


然后即可查看目前已经写入的函数和类：

In [13]:
from manual_ensemble import *

In [14]:
VotingClassifier_threshold?

Init signature:
VotingClassifier_threshold(
    estimators,
    voting='hard',
    weights=None,
    thr=0.5,
)
Docstring:     
Base class for all estimators in scikit-learn.

Notes
-----
All estimators should specify all the parameters that can be set
at the class level in their ``__init__`` as explicit keyword
arguments (no ``*args`` or ``**kwargs``).
File:           d:\work\jupyter\telco\正式课程\manual_ensemble.py
Type:           type
Subclasses:     


In [15]:
train_cross?

Signature:
train_cross(
    X_train,
    y_train,
    X_test,
    estimators,
    test_size=0.2,
    n_splits=5,
    random_state=12,
    blending=False,
)
Docstring:
Stacking融合过程一级学习器交叉训练函数

:param X_train: 训练集特征
:param y_train: 训练集标签
:param X_test: 测试集特征
:param estimators: 一级学习器，由(名称,评估器)组成的列表
:param n_splits: 交叉训练折数
:param test_size: blending过程留出集占比
:param random_state: 随机数种子
:param blending: 是否进行blending融合

:return：交叉训练后创建oof训练数据和测试集平均预测结果
File:      d:\work\jupyter\telco\正式课程\manual_ensemble.py
Type:      function


准备完毕后，正式进入到本节内容。

> 关于manual_ensemble.py内部代码结构，将在函数编写时逐渐完善。

## 十、Stacking融合优化函数

&emsp;&emsp;Stacking本身的流程其实并不复杂，无非就是训练一级学习器并创建oof训练数据，然后带入元学习器进行第二轮的训练。而在实际工程化实践过程中，较为复杂的是如何高效的进行一级学习器的训练，以及如何进行元学习器的优化。接下来，我们围绕这两个环节的优化策略来进行整理和函数编写。

### 1.交叉训练过程超参数自动优化过程

- 两种交叉训练策略回顾

&emsp;&emsp;对于一级学习器的训练，在上一小节我们介绍了目前通用的两种较为有效的训练策略：其一是在全部数据集上训练一组一级学习器并进行超参数优化，然后组内共用一组超参数来进行交叉训练，即训练过程如下：

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221008113027435.png" alt="image-20221008113027435" style="zoom:50%;" />

这其实是最通用的流程，并且并不难执行。一般来说，我们在机器学习建模的初期，都是需要围绕单模进行模型训练和优化的，此时我们只需要把训练好的单独模型组成estimators，然后带入train_cross输出oof数据集即可。

&emsp;&emsp;接下来我们重点讨论是第二种方法如何高效实现。上一小节我们介绍了基于该交叉训练过程实现的基本过程，相比之前的交叉训练过程，本方法重点在于需要实现组内每个模型每次训练的超参数优化，其具体实现过程如下：

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221008113103211.png" alt="image-20221008113103211" style="zoom:50%;" />

&emsp;&emsp;其实从上一小节的融合结果我们已经能明显看出一级学习器的不同训练过程对结果的影响程度。尽管很多时候出于建模流程效率考虑，会选择在全数据集上训练一组超参数，然后在固定这组超参数的情况下进行交叉训练，但交叉训练过程单独训练每个模型的超参数的优势是确实存在的。交叉训练过程中组内超参数独立训练，能有效提升oof数据集中信息隔离效果，进而提高Stacking泛化能力。

- 基于贝叶斯优化的交叉训练策略

&emsp;&emsp;而具体如何实现，我们可以执行类似Part 4.3.4中单独创建五个不同的训练集子集、然后每个模型都手动优化得到5个模型，然后再创建train_oof数据集。不过该策略耗时较长，需要耗费大量的时间一个个模型进行超参数优化。当然，我们也可以采用一种更加自动化的方法来执行，即在train_cross函数执行过程中，fit过程直接带入网格搜索评估器，此时输出结果就将是一个个超参数优化后的评估器。

&emsp;&emsp;不过fit网格搜索评估器的方法会非常耗时，对于很多超参数范围较大的集成算法，单轮的搜索就需要至少十几甚至几十分钟，更何况为了让5个不同的模型在各自不同的训练数据集上自适应的搜索出最佳参数，参数空间也需要设置一个较大的范围，因此在train_cross过程中执行fit网格搜索评估器的操作几乎不可行。

&emsp;&emsp;那能否替换成别的优化器呢？其实相比之下，贝叶斯优化器会更适用于当前情况。贝叶斯优化器有较快的执行速度、并且低使用门槛、高效果上限，少量迭代即可初见成效，大量迭代也可以确保效果，至于具体迭代多少次，完全可以根据当前算力情况来决定。这里我们仍然可以考虑使用hyperOPT优化器来执行。但唯一的问题就是，但是，原生的hyperOPT优化过程是多个函数的计算流（定义超参数空间——定义目标函数——定义优化函数——执行优化过程），我们需要将这一过程封装成一个sklearn的评估器，才可借助一个fit语句就完成这一整个流程，进而才可带入train_cross中进行自动超参数优化。

&emsp;&emsp;接下来，我们就尝试借助hyperOPT来执行超参数自动优化的交叉训练。

### 2.借助hyperOPT完成交叉训练过程超参数自动优化过程

&emsp;&emsp;我们首先从决策树模型入手，将原先hyperOPT过程封装成一个评估器，并测试fit调用结果。

- 决策树模型超参数优化评估器

&emsp;&emsp;这里我们先回顾下整个决策树模型的hyperOPT的优化过程，首先是参数搜索空间的创建：

In [16]:
tree_params_space = {'tree_max_depth': hp.choice('tree_max_depth', np.arange(2, 20).tolist()), 
                     'tree_min_samples_split': hp.choice('tree_min_samples_split', np.arange(2, 15).tolist()), 
                     'tree_min_samples_leaf': hp.choice('tree_min_samples_leaf', np.arange(1, 15).tolist()), 
                     'tree_max_leaf_nodes': hp.choice('tree_max_leaf_nodes', np.arange(2, 51).tolist())}

&emsp;&emsp;然后定义目标函数。在这里的目标函数定义时，为了方便后续直接带入搜索得到的最佳参数在测试集上进行验证，目标函数获取参数取值的方式分为两种，其一是训练过程，直接从params_space传入参数，而在测试过程，则直接传入搜索后的params_best，并将hp.choice对象得到的索引转化为具体数值。不同的训练过程通过train参数控制，这里我们先看train=True的情况，也就是训练过程，然后再讨论测试时索引值和具体数值的转化关系：

In [19]:
def hyperopt_tree(params, train=True):
    # 读取参数
    if train == True:
        max_depth = params['tree_max_depth']
        min_samples_split = params['tree_min_samples_split']
        min_samples_leaf = params['tree_min_samples_leaf']
        max_leaf_nodes = params['tree_max_leaf_nodes']
    else: 
        max_depth = params['tree_max_depth'] + 2
        min_samples_split = params['tree_min_samples_split'] + 2
        min_samples_leaf = params['tree_min_samples_leaf'] + 1
        max_leaf_nodes = params['tree_max_leaf_nodes'] + 2
        
    # 实例化模型
    tree = DecisionTreeClassifier(max_depth=max_depth, 
                                  min_samples_split=min_samples_split, 
                                  min_samples_leaf=min_samples_leaf, 
                                  max_leaf_nodes=max_leaf_nodes)
    
    if train == True:
        res = -cross_val_score(tree, X_train_OE, y_train).mean()
    else:
        res = tree.fit(X_train_OE, y_train)
    
    return res

In [20]:
def param_hyperopt_tree(max_evals):
    params_best = fmin(fn = hyperopt_tree,
                       space = tree_params_space,
                       algo = tpe.suggest,
                       max_evals = max_evals, 
                       rstate=np.random.RandomState(9))    
    
    return params_best

&emsp;&emsp;接下来，测试优化能否顺利运行：

In [21]:
tree_params_best = param_hyperopt_tree(1000)

100%|███████████████████████████████████████████| 1000/1000 [00:48<00:00, 20.68trial/s, best loss: -0.7962873770820791]


In [18]:
tree_params_best

{'tree_max_depth': 3,
 'tree_max_leaf_nodes': 25,
 'tree_min_samples_leaf': 9,
 'tree_min_samples_split': 1}

这里需要注意，在定义目标函数的时候，仍然是区分了目标函数的训练状态和测试状态，训练状态（train=True）是默认状态，作为超参数搜索时的目标函数时使用，而train=Fasle时则为测试状态，此时函数用于带入搜索出来的超参数，来直接输出最终的最优模型：

In [79]:
hyperopt_tree(tree_params_best, train=False)

DecisionTreeClassifier(max_depth=5, max_leaf_nodes=27, min_samples_leaf=10,
                       min_samples_split=3)

In [80]:
clf = hyperopt_tree(tree_params_best, train=False)

然后即可进一步测试模型在测试集上的评分：

In [82]:
clf.score(X_test_OE, y_test)

0.7768313458262351

而训练状态和测试状态的重要区别，就在于参数的导入。对于hyperOPT来说，hp.choice的搜索结果其实是原始参数取值列表的索引值，例如max_depth：3，其实代表的是原始参数空间中'tree_max_depth': hp.choice('RF_max_depth', np.arange(2, 20).tolist())的第3个值，也就是2+3=5:

In [4]:
np.arange(2, 20)

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])

In [5]:
np.arange(2, 20)[3]

5

因此目标函数在定义train=False的代码时，对于整数列表的数值提取，只需要用得到的索引值+列表初始值即可。再比如max_leaf_nodes的最佳值索引是27,则真实值为2+27=29。当然，对于字符串列表，则需要直接把字符串完整列表带入进行索引。

&emsp;&emsp;而通过目标函数不同模式的改写，能极大程度提高目标函数的复用率，使代码更加简洁，并且测试模式下输出的最佳模型，也是后面要用到的关键对象。

&emsp;&emsp;在熟悉了改写后的hyperOPT后，接下来将这一整个流程封装为一个评估器，方便后续直接使用fit方法调用：

In [25]:
class tree_cascade(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, tree_params_space, max_evals=1000):
        self.tree_params_space = tree_params_space
        self.max_evals = max_evals
        
    def fit(self, X, y):
        def hyperopt_tree(params, train=True):
            # 读取参数
            if train == True:
                max_depth = params['tree_max_depth']
                min_samples_split = params['tree_min_samples_split']
                min_samples_leaf = params['tree_min_samples_leaf']
                max_leaf_nodes = params['tree_max_leaf_nodes']
            else: 
                max_depth = params['tree_max_depth'] + 2
                min_samples_split = params['tree_min_samples_split'] + 2
                min_samples_leaf = params['tree_min_samples_leaf'] + 1
                max_leaf_nodes = params['tree_max_leaf_nodes'] + 2

            # 实例化模型
            tree = DecisionTreeClassifier(max_depth=max_depth, 
                                          min_samples_split=min_samples_split, 
                                          min_samples_leaf=min_samples_leaf, 
                                          max_leaf_nodes=max_leaf_nodes, 
                                          random_state=12)

            if train == True:
                res = -cross_val_score(tree, X, y).mean()
            else:
                res = tree.fit(X, y)

            return res

        def param_hyperopt_tree(max_evals):
            params_best = fmin(fn = hyperopt_tree,
                               space = self.tree_params_space,
                               algo = tpe.suggest,
                               max_evals = max_evals, 
                               rstate=np.random.RandomState(9))    

            return params_best
        
        tree_params_best = param_hyperopt_tree(self.max_evals)
        self.clf = hyperopt_tree(tree_params_best, train=False)
        return self
    
    def predict_proba(self, X):
        res_proba = self.clf.predict_proba(X)
        return res_proba
    
    def predict(self, X):
        res = self.clf.predict(X)
        return res
    
    def score(self, X, y):
        res = self.clf.score(X, y)
        return res

&emsp;&emsp;这里需要注意，在定义评估器的过程中，围绕每个评估器都设置了一个默认的迭代次数，我们可以根据实际算力情况来对其进行灵活调整，迭代次数越多、最终结果也将更加精确。此外，需要注意tree_cascade类内部fit函数的X和y的传递过程，由于fit函数内部是有X和y这一组局部变量的，所以在fit函数内部的hyperopt_tree函数就可以对其直接引入，而不用以参数形式引入。并且，hyperopt_tree函数内部目标函数的输出结果是（五折）交叉验证结果，用以提高搜索结果的泛化能力。

&emsp;&emsp;然后测试评估器能否顺利执行：

In [26]:
tree_hyper = tree_cascade(tree_params_space)

In [27]:
tree_hyper.fit(X_train_OE, y_train)

100%|███████████████████████████████████████████| 1000/1000 [00:49<00:00, 20.11trial/s, best loss: -0.7962873770820791]


tree_cascade(tree_params_space={'tree_max_depth': <hyperopt.pyll.base.Apply object at 0x00000288CEF8C9D0>,
                                'tree_max_leaf_nodes': <hyperopt.pyll.base.Apply object at 0x00000288D04FA460>,
                                'tree_min_samples_leaf': <hyperopt.pyll.base.Apply object at 0x00000288CAD32100>,
                                'tree_min_samples_split': <hyperopt.pyll.base.Apply object at 0x00000288CAD32040>})

In [28]:
tree_hyper.predict(X_test_OE)

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [29]:
tree_hyper.score(X_test_OE, y_test)

0.7768313458262351

In [66]:
tree_hyper = tree_cascade(tree_params_space, max_evals=2000).fit(X_train_OE, y_train)

100%|███████████████████████████████████████████| 2000/2000 [01:51<00:00, 18.02trial/s, best loss: -0.7962873770820791]


In [67]:
tree_hyper.score(X_test_OE, y_test)

0.7768313458262351

&emsp;&emsp;这里需要注意，hyperOPT搜索效果其实也是有“上限”的，也就是说当hyperOPT的搜索会存在当迭代次数超过某个数值时，增加迭代次数并不会提升模型效果。从原理层面来说，是因为新的超参数组合结果并不会影响此前的估计，而具体达到效果上限需要多少次迭代，则和数据量、模型复杂度、甚至是随机数种子都有很大关系。这里，对于当前数据集来说，决策树模型迭代1000次是可以达到效果上限的，而1000次的迭代也仅需要1分钟不到的时间，因此，决策树的优化评估器一般无须调整迭代次数。

&emsp;&emsp;但对于更加复杂的集成学习来说，这个到达上限所需的迭代次数可能会很高，单次搜索达到上限的时间可能会很长，因此，在后续交叉训练过程中，我们是否要每次模型训练都令其达到这个上限，也需要根据实际情况来决定。

&emsp;&emsp;至此，我们就完成了决策树模型的超参数优化评估器的封装，接下来我们需要将其写入manual_ensemble.py模块中。当然，除了决策树模型外，我们还需要类似的改写另外两个模型的TPE优化过程。

- 随机森林超参数优化评估器

In [70]:
RF_params_space = {'RF_min_samples_leaf': hp.choice('RF_min_samples_leaf', np.arange(1, 20).tolist()), 
                   'RF_min_samples_split': hp.choice('RF_min_samples_split', np.arange(2, 20).tolist()), 
                   'RF_max_depth': hp.choice('RF_max_depth', np.arange(2, 20).tolist()), 
                   'RF_max_leaf_nodes': hp.choice('RF_max_leaf_nodes', np.arange(20, 200).tolist()), 
                   'RF_n_estimators': hp.choice('RF_n_estimators', np.arange(20, 200).tolist()), 
                   'RF_max_samples': hp.uniform('RF_max_samples', 0.2, 0.8)}

In [78]:
class RF_cascade(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, RF_params_space, max_evals=500):
        self.RF_params_space = RF_params_space
        self.max_evals = max_evals
        
    def fit(self, X, y):
        def hyperopt_RF(params, train=True):
            # 读取参数
            if train == True:
                min_samples_leaf = params['RF_min_samples_leaf']
                min_samples_split = params['RF_min_samples_split']
                max_depth = params['RF_max_depth']
                max_leaf_nodes = params['RF_max_leaf_nodes']
                n_estimators = params['RF_n_estimators']
                max_samples = params['RF_max_samples']
            else: 
                min_samples_leaf = params['RF_min_samples_leaf'] + 1
                min_samples_split = params['RF_min_samples_split'] + 2
                max_depth = params['RF_max_depth'] + 2
                max_leaf_nodes = params['RF_max_leaf_nodes'] + 20
                n_estimators = params['RF_n_estimators'] + 20
                max_samples = params['RF_max_samples']
            # 实例化模型
            RF = RandomForestClassifier(min_samples_leaf = min_samples_leaf, 
                                        min_samples_split = min_samples_split,
                                        max_depth = max_depth, 
                                        max_leaf_nodes = max_leaf_nodes, 
                                        n_estimators = n_estimators, 
                                        max_samples = max_samples)
            if train == True:
                res = -cross_val_score(RF, X, y).mean()
            else:
                res = RF.fit(X, y)

            return res

        def param_hyperopt_RF(max_evals):
            params_best = fmin(fn = hyperopt_RF,
                               space = self.RF_params_space,
                               algo = tpe.suggest,
                               max_evals = max_evals)    

            return params_best
        
        RF_params_best = param_hyperopt_RF(self.max_evals)
        self.clf = hyperopt_RF(RF_params_best, train=False)
        return self
    
    def predict_proba(self, X):
        res_proba = self.clf.predict_proba(X)
        return res_proba
    
    def predict(self, X):
        res = self.clf.predict(X)
        return res
    
    def score(self, X, y):
        res = self.clf.score(X, y)
        return res

In [72]:
RF_hyper = RF_cascade(RF_params_space)

In [58]:
RF_hyper.fit(X_train_OE, y_train)

100%|█████████████████████████████████████████████| 500/500 [07:39<00:00,  1.09trial/s, best loss: -0.8087839726498667]


RF_cascade(RF_params_space={'RF_max_depth': <hyperopt.pyll.base.Apply object at 0x000002AAD8A233A0>,
                            'RF_max_leaf_nodes': <hyperopt.pyll.base.Apply object at 0x000002AAD8A23A00>,
                            'RF_max_samples': <hyperopt.pyll.base.Apply object at 0x000002AAD8A33FA0>,
                            'RF_min_samples_leaf': <hyperopt.pyll.base.Apply object at 0x000002AAD8A219D0>,
                            'RF_min_samples_split': <hyperopt.pyll.base.Apply object at 0x000002AAD8A21A90>,
                            'RF_n_estimators': <hyperopt.pyll.base.Apply object at 0x000002AAD8A30C70>})

In [153]:
RF_hyper.predict(X_test_OE)

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [154]:
RF_hyper.score(X_test_OE, y_test)

0.7864849517319704

&emsp;&emsp;当然，其实对于当前数据集来说，随机森林迭代500次并没有达到效果的“上限”，这里我们可以尝试迭代1000次查看结果：

In [73]:
RF_hyper = RF_cascade(RF_params_space, max_evals=1000).fit(X_train_OE, y_train)

100%|███████████████████████████████████████████| 1000/1000 [14:57<00:00,  1.11trial/s, best loss: -0.8093528711906195]


In [84]:
RF_hyper.score(X_test_OE, y_test)

0.7881885292447472

能够发现，增加迭代次数后模型效果有了更进一步提升，但1000次的迭代是否已经达到了TPE搜索的效果“上限”？我们是否应该测试更多次迭代的结果？并且，既然发现更多次的迭代能够有效果上的进一步提升，我们是否应该修改默认迭代次数？

&emsp;&emsp;需要说明的是，这里设置默认迭代500次是为了便于后续交叉训练的快速执行，方便快速测试跑通代码（一次随机森林模型优化需要8分钟，五轮交叉训练就需要40分钟，然后还要加上其他模型的训练时间），以及后续进行更大范围不同模型组合和特征组合的效果验证。当然，在本小节的最后，在跑通了整个流程后，会有一次更多迭代次数的运行过程，以测试基于TPE交叉训练的效果极限。

> 此外，估计类的优化算法尽管执行效率很高，但精度其实不如枚举的网格搜索。不过网格搜索所需时间远远高于TPE搜索，如Part 4.2中列举的过程，单模的高精度搜索就需要3-4个小时。

> 对于估计类优化算法，另外一个需要注意的地方就是，优化效果的提升会伴随着迭代次数增加而递减，例如随机森林的前500次迭代就已经达到了0.7864，而后面再增加500次迭代，也仅提升了0.2%准确率。换而言之，算力的消耗相比模型效果的提升，“性价比”是逐渐降低的。

- 逻辑回归超参数优化评估器

In [58]:
lr_params_space = {'lr_C': hp.uniform('lr_C', 0, 1), 
                   'lr_penalty': hp.choice('lr_penalty', ['l1', 'l2']), 
                   'lr_thr': hp.uniform('lr_thr', 0, 1)}

In [61]:
class lr_cascade(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, lr_params_space, max_evals=20):
        self.lr_params_space = lr_params_space
        self.max_evals = max_evals
        
    def fit(self, X, y):
        def hyperopt_lr(params, train=True):
            # 读取参数
            if train == True:
                C = params['lr_C']
                penalty = params['lr_penalty']
                thr = params['lr_thr']
            else: 
                C = params['lr_C']
                penalty = ['l1', 'l2'][params['lr_penalty']]
                thr = params['lr_thr']
            # 实例化模型
            lr = logit_threshold(C = C,  
                                 thr = thr, 
                                 penalty = penalty, 
                                 solver = 'saga', 
                                 max_iter = int(1e6))
            
            if train == True:
                res = -cross_val_score(lr, X, y).mean()
            else:
                res = lr.fit(X, y)

            return res

        def param_hyperopt_lr(max_evals):
            params_best = fmin(fn = hyperopt_lr,
                               space = self.lr_params_space,
                               algo = tpe.suggest,
                               max_evals = max_evals, 
                               rstate=np.random.RandomState(9))    

            return params_best
        
        lr_params_best = param_hyperopt_lr(self.max_evals)
        self.clf = hyperopt_lr(lr_params_best, train=False)
        return self
    
    def predict_proba(self, X):
        res_proba = self.clf.predict_proba(X)
        return res_proba
    
    def predict(self, X):
        res = self.clf.predict(X)
        return res
    
    def score(self, X, y):
        res = self.clf.score(X, y)
        return res

&emsp;&emsp;对于逻辑回归的超参数优化过程，由于逻辑回归本身特性导致不需要太多次的搜索就能得到一组较为稳定的结果，但saga优化器运行效率有限，每次计算都需要较长时间，因此默认迭代次数是20次。此外，对于当前数据集来说，逻辑回归约迭代50次左右能达到TPE搜索效果上限。

In [62]:
lr_hyper = lr_cascade(lr_params_space)

In [63]:
lr_hyper.fit(X_train_OE, y_train)

100%|████████████████████████████████████████████████| 20/20 [02:24<00:00,  7.21s/trial, best loss: -0.788717174106247]


lr_cascade(lr_params_space={'lr_C': <hyperopt.pyll.base.Apply object at 0x00000288D2BE9460>,
                            'lr_penalty': <hyperopt.pyll.base.Apply object at 0x00000288D2BE95B0>,
                            'lr_thr': <hyperopt.pyll.base.Apply object at 0x00000288D2BE9490>})

In [64]:
lr_hyper.predict(X_test_OE)

array([0, 0, 0, ..., 0, 0, 0])

In [65]:
lr_hyper.score(X_test_OE, y_test)

0.7773992049971608

- 基于超参数优化评估器的train_cross过程

&emsp;&emsp;在定义了三个超参数优化评估器后，我们即可将其带入交叉训练函数中进行超参数搜索，并最终输出oof数据集。这里需要注意，此时带入训练的不再是一个个简单的模型，而是封装为评估器的超参数优化器，因此在实际训练的过程、每次划分完训练集和验证集之后，模型都会在给定的训练集上进行超参数优化和模型训练，然后再在测试集上输出预测结果。我们可以看下由此输出的oof数据集效果如何：

In [59]:
lr_hyper = lr_cascade(lr_params_space)
tree_hyper = tree_cascade(tree_params_space)
RF_hyper = RF_cascade(RF_params_space)

estimators = [('lr', lr_hyper), ('tree', tree_hyper), ('rf', RF_hyper)]

In [60]:
train_oof, test_predict = train_cross(X_train_OE, y_train, X_test_OE, estimators=estimators)

100%|█████████████████████████████████████████████| 500/500 [06:59<00:00,  1.19trial/s, best loss: -0.8090352091988754]


由此，我们自动完成了交叉训练过程中的自动超参数搜索。当然，要自动进行多组模型超参数搜索，还是需要花费一段运行时间的。为了方便后续计算运算时间对于结果提升的“能效比”，我们从此处开始需要统计不同流程得出一个融合结果所需要耗费的计算时间，以上述过程为例，在默认参数设置情况下，一级学习器的训练及oof数据集的创建约需要45分钟。

&emsp;&emsp;接下来进行简单测试，以逻辑回归作为元学习器，输出最终融合结果：

In [61]:
# 设置超参数空间
logistic_param = [
    {'thr': np.arange(0.1, 1, 0.1).tolist(), 'penalty': ['l1'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['saga']}, 
    {'thr': np.arange(0.1, 1, 0.1).tolist(), 'penalty': ['l2'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['lbfgs', 'newton-cg', 'sag', 'saga']}, 
]

# 实例化相关评估器
logistic_final = logit_threshold(max_iter=int(1e6))
    
# 执行网格搜索
lfg = GridSearchCV(estimator = logistic_final,
                   param_grid = logistic_param,
                   scoring='accuracy',
                   n_jobs = 15).fit(train_oof, y_train)

lfg.score(train_oof, y_train), lfg.score(test_predict, y_test)

(0.8347216963271488, 0.7904599659284497)

In [62]:
lfg.best_params_

{'C': 1.0, 'penalty': 'l1', 'solver': 'saga', 'thr': 0.5}

能够发现，相比单模结果，Stacking融合结果结果有了进一步提升。

|得分|训练集|测试集|
|:--:|:--:|:--:|
|Tree单模|0.7962|0.7768|
|RF单模|0.8087|0.7864|
|LR单模|0.7887|0.7773|
|final-lr|0.8347|0.7904|

### 2.元学习器自动优化函数

- 函数定义

&emsp;&emsp;既然是为了更高效率、更高精度的执行Stacking融合，除了定义超参数自动优化的交叉训练函数外，还有非常重要的一环，那就是定义元学习器的自动优化函数。在上一小节中，我们总结了元学习器的模型训练流程，即逻辑回归&决策树单模优化、元学习器交叉训练、元学习器Bagging集成等方案，然后从这些模型结果中则优输出。我们可以总结这个流程如下：

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221008155028693.png" alt="image-20221008155028693" style="zoom:50%;" />

接下来，我们元学习器优化流程封装为一个函数final_model_opt，在输入元学习器组和超参数空间组的情况下，自动筛选最优元学习器建模流程，并输出在该流程下测试集上的预测结果。函数定义过程如下：

In [106]:
def final_model_opt(final_model_l, param_space_l, X, y, test_predict):
    """
    Stacking元学习器自动优化与预测函数
    
    :param final_model_l: 备选元学习器组成的列表
    :param param_space_l: 备选元学习器各自超参数搜索空间组成的列表
    :param X: oof_train训练集特征
    :param y: oof_train训练集标签
    :param test_predict: 一级评估器输出的测试集预测结果
    
    :return：多组元学习器在oof_train上的最佳评分，以及最佳元学习器在test_predict上的预测结果
    """
    
    # 不同组元学习器结果存储列表
    # res_l用于存储模型在训练集上的评分
    res_l = np.zeros(len(final_model_l)).tolist()
    # test_predict_l用于存储模型在测试集test_predict上的预测结果
    test_predict_l = np.zeros(len(final_model_l)).tolist()
    
    for i, model in enumerate(final_model_l):
        # 输出元学习器单模预测结果
        # 执行网格搜索
        model_grid = GridSearchCV(estimator = model,
                                  param_grid = param_space_l[i],
                                  scoring='accuracy',
                                  n_jobs = 15)
        model_grid.fit(X, y)
        # 记录单模最佳模型，方便后续作为Bagging的基础评估器
        res1_best_model = model_grid.best_estimator_
        # 测试在训练oof数据集上的准确率
        res1 = model_grid.score(X, y)
        # 输出单模在test_predict上的预测结果
        res1_test_predict = model_grid.predict_proba(test_predict)[:, 1]
        
        # 输出元学习器交叉训练预测结果
        res2_temp = np.zeros(y.shape[0])
        res2_test_predict = np.zeros(test_predict.shape[0])
        # 交叉训练过程附带网格搜索以提升精度
        folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=12)
        for trn_idx, val_idx in folds.split(X, y):
            model_grid = GridSearchCV(estimator = model,
                                      param_grid = param_space_l[i],
                                      scoring='accuracy',
                                      n_jobs = 15)
            model_grid.fit(X.loc[trn_idx], y.loc[trn_idx])
            res2_temp += model_grid.predict_proba(X)[:, 1] / 10
            # 记录测试集上的预测结果
            res2_test_predict += model_grid.predict_proba(test_predict)[:, 1] / 10
        # 交叉训练模型组评分
        res2 = accuracy_score((res2_temp >= 0.5) * 1, y)

        # 元学习器的Bagging过程
        bagging_param_space = {"n_estimators": range(10, 21), 
                               "max_samples": np.arange(0.1, 1.1, 0.1).tolist()}
        
        bagging_final = BaggingClassifier(res1_best_model)
        BG = GridSearchCV(bagging_final, bagging_param_space, n_jobs=15).fit(X, y)
        # Bagging元学习器评分
        res3 = BG.score(X, y)
        # Bagging元学习器在测试集上评分
        res3_test_predict = BG.predict_proba(test_predict)[:, 1]
        
        # 三组模型评分组成列表
        res_l_temp = [res1, res2, res3]
        # 三组模型在测试集上预测结果组成列表
        test_predict_l_temp = [res1_test_predict, res2_test_predict, res3_test_predict]
        # 挑选评分最高模型
        best_res = np.max(res_l_temp)
        # 挑选评分最高模型输出的测试集概率预测结果
        best_test_predict = test_predict_l_temp[np.argmax(res_l_temp)]
        # 将最佳模型写入res_l对应位置
        res_l[i] = best_res
        # 将最佳模型在测试集上的评分写入test_predict_l
        test_predict_l[i] = best_test_predict
        
    # 再从res_l中选取训练集上最佳评分
    best_res_final = np.max(res_l) 
    # 根据训练集上的最佳评分，选取挑选最佳测试集预测结果
    best_test_predict_final = test_predict_l[np.argmax(res_l)]
    
    return best_res_final, best_test_predict_final

&emsp;&emsp;这里我们在res1、res2、res3的创建和选取上，增加了更加一层备选，即增加了元学习器的决策树和逻辑回归两个模型备选的过程，即实际上我们是在res1_lr、res2_lr、res3_lr和res1_tree、res2_tree、res3_tree。当然，为了整体迭代方便，这里我们采用了两层筛选的机制，即先从res1_lr、res2_lr、res3_lr中挑选最佳结果，然后和res1_tree、res2_tree、res3_tree最佳结果进行比较

- 函数性能及可拓展性讨论

&emsp;&emsp;并且需要注意，在大多数情况下，真实的测试集标签其实是不可知的，我们更无法借助测试集的标签取值来筛选模型或者超参数。测试集作为需要被预测的对象，final_model_opt函数最终只能够输出测试集的概率预测结果。当然，如果当前测试集是存在标签的，那么也可以根据best_test_predict_final测试最终融合结果在测试集上的表现。但从通用性角度考虑，final_model_opt函数只提供元评估器优化后的预测结果。此外，对于元学习器（包括元学习器的Bagging评估器）的优化器采用的是更高精度的网格搜索，以便能够高效、精准的输出预测结果。当然，这里的优化器也可以换成贝叶斯优化器。

&emsp;&emsp;而原学习器的选择方面，其实在不引入特征增强的情况下，并没有额外的选择，因此其实也可以考虑把原学习器写死在函数内部。而对于评分函数，目前是使用默认准确率，当然无论是函数内部的hyperOPT优化过程还是sklearn的网格搜索优化器，都可以非常便捷的调整评分函数。

- 函数使用

&emsp;&emsp;接下来测试函数效果，首先定义函数的元学习器列表。不同于train_cross的一级学习器列表，元学习器列表不需要将单独的模型改写成元组再组成列表，直接实例化模型后组成列表即可：

In [98]:
lr = logit_threshold()
tree = DecisionTreeClassifier()
final_model_l = [lr, tree]

然后是超参数空间列表的创建过程，这里只需要各超参数空间和元学习器的模型顺序保持一致即可：

In [109]:
lr_final_param = [{'thr': np.arange(0.1, 1.1, 0.1).tolist(), 'penalty': ['l1'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['saga']}, 
                  {'thr': np.arange(0.1, 1.1, 0.1).tolist(), 'penalty': ['l2'], 'C': np.arange(0.1, 1.1, 0.1).tolist(), 'solver': ['lbfgs', 'newton-cg', 'sag', 'saga']}]

tree_final_param = {'max_depth': np.arange(2, 16, 1).tolist(), 
                    'min_samples_split': np.arange(1, 5, 1).tolist(), 
                    'min_samples_leaf': np.arange(1, 4, 1).tolist(), 
                    'max_leaf_nodes':np.arange(6, 30, 1).tolist()}

param_space_l = [lr_final_param, tree_final_param]

接下来测试元学习器优化函数的最终效果：

In [113]:
best_res_final, best_test_predict_final = final_model_opt(final_model_l, param_space_l, train_oof, y_train, test_predict)

In [139]:
accuracy_score((best_test_predict_final >= 0.5) * 1, y_test)

0.7961385576377058

在引入元学习器优化过程后，结果有了明显提升：

|得分|训练集|测试集|
|:--:|:--:|:--:|
|Tree单模|0.7962|0.7768|
|RF单模|0.8087|0.7864|
|LR单模|0.7887|0.7773|
|final-lr|0.8347|0.7904|
|meta-opt|-|0.7961|

至此，我们就完整构建了元学习器的训练和优化流程。本部分的final_model_opt函数，以及原学习器的超参数空间都需要写入manual_emsemble.py文件中。

### 3.自动Stacking模型融合流程

&emsp;&emsp;在定义了自动交叉训练函数和自动元学习器优化函数之后，即可将二者串联使用，来完成自动Stacking模型融合过程。其基本实现过程流程如下：

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221009212324752.png" alt="image-20221009212324752" style="zoom:50%;" />

该流程能够在极少量代码的情况下快速实现高精度模型融合，并且我们可以非常灵活的通过调整一级学习器的迭代次数来平衡融合效率和融合效果。当我们需要尽可能提升模型结果时，可以尽可能增加迭代次数，令其逼近TPE优化效果上限，以提升最终融合结果。而如果我们是希望快速对比测试不同模型的组合效果、或者对比不同特征分配的情况下融合效果，则可以相对设置更少的迭代次数，来进行快速的运行，此时尽管效果有限，但用于对比测试足以。

&emsp;&emsp;接下来，我们提高一级学习器交叉训练的迭代次数，尝试借助这个自动Stacking的流程获得一个更好的效果：

In [110]:
lr_hyper = lr_cascade(lr_params_space, max_evals=50)
tree_hyper = tree_cascade(tree_params_space)
RF_hyper = RF_cascade(RF_params_space, max_evals=1000)

estimators = [('lr', lr_hyper), ('tree', tree_hyper), ('rf', RF_hyper)]

In [111]:
train_oof, test_predict = train_cross(X_train_OE, y_train, X_test_OE, estimators=estimators)

100%|███████████████████████████████████████████| 1000/1000 [12:51<00:00,  1.30trial/s, best loss: -0.8095083022087932]


元学习器的交叉训练约用时1个半小时。其实相比手动训练模型过程，时间还是有大幅缩短的。接下来将元学习器训练数据带入元学习器优化函数，测试效果：

In [112]:
lr = logit_threshold()
tree = DecisionTreeClassifier()
final_model_l = [lr, tree]

In [113]:
best_res_final, best_test_predict_final = final_model_opt(final_model_l, param_space_l, train_oof, y_train, test_predict)

In [137]:
accuracy_score((best_test_predict_final >= 0.5) * 1, y_test)

0.7978421351504826

|得分|训练集|测试集|
|:--:|:--:|:--:|
|Tree单模|0.7962|0.7768|
|RF单模|0.8087|0.7864|
|LR单模|0.7887|0.7773|
|final-lr|0.8347|0.7904|
|meta-opt|-|0.7961|
|final-opt|-|0.7978|

最终结果超过此前最好单模成绩（0.7955），低于此前手动融合的模型结果（0.8001）。当然，整体结果仍然是可用的结果，而结果的小幅随机扰动其实会跟优化器、随机数种子有较大关系。而一个自动化的流程，将在大范围自动搜索的过程中发挥更大的作用，也将在后续替换其他集成算法时展示出更优秀的优化结果。

&emsp;&emsp;最后，简单探讨下关于暴力计算之于模型融合的实际价值。

&emsp;&emsp;其实从本节开始，会有越来越多的长时间代码运算过程，用于复杂优化过程、特征筛选和创建、级联优化等，这也就是所谓的“暴力计算”。而其实暴力计算对于机器学习来说至关重要，很多时候一定量的计算时间，也是一个更好结果的基本保障，人们戏称这个过程为“炼丹”。尽管很多时候我们都希望能够有一个简洁美观的公式，通过一系列高效的计算就能迅速得出一个非常好的结论，但实际上，作为后验的算法，机器学习在很多时候还是需要暴力计算来算出一个还不错的结果。目前来说，无论是企业应用还是算法竞赛，我们都能常常看到复杂代码和暴力计算的身影。其实早在2006年奈飞组织的第一场数据科学竞赛，第一名的队伍就采用了107个算法融合的策略，总训练时间长达2000小时。当然，伴随着算法的不断推陈出新以及特征工程、模型融合的技术更新，模型优化和训练速度大幅提升，一个模型跑2000小时也早已成为历史，但暴力计算的之于深度学习和机器学习仍然非常重要。

&emsp;&emsp;而在课上，围绕“通过暴力计算得出更好结果”这一议题，也将提供更多的实例。不过鉴于个人用户的算例有限，大多数暴力计算过程都会控制在3小时内，确保实战中可以运行。